In [1]:
import leafmap

In [2]:
import planetary_computer as pc
import pystac_client
from pystac.extensions.projection import ProjectionExtension as proj
import geopandas as gpd
import json

In [3]:
aoi = 'fiji_suva'
local =  gpd.read_file(aoi + '.geojson')
area_of_interest = local.geometry[0]

In [4]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

search = catalog.search(
    intersects=area_of_interest,
    datetime="2020-01-01/2022-03-30",
    collections=["sentinel-2-l2a"],
    limit=500,  # fetch items in batches of 500
    query={"eo:cloud_cover": {"lt": 1.0}}, #cloudcover_percentage
)

raw_items = search.get_items()

#least cloud cover
raw_items = sorted(raw_items, key=lambda item: item.properties['eo:cloud_cover'])[0:1]

#sign
items = [pc.sign(item).to_dict() for item in raw_items]

#item/epsg
item = next(search.get_items())
epsg = proj.ext(item).epsg


In [5]:
print(item.datetime)

2021-07-07 22:19:39.024000+00:00


In [9]:
m = leafmap.Map(zoom=1, google_map="ROADMAP")
m.add_stac_layer(
    collection="sentinel-2-l2a",
    item=item.id,
    assets=["B04", "B03", "B02"], #red, green, blue
    color_formula="Gamma RGB 3.7 Saturation 1.5 Sigmoidal RGB 15 0.35",
    nodata=0,
    rescale="0,12000",

)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [13]:
m = leafmap.Map(zoom=1, google_map="ROADMAP")
m.add_stac_layer(
    collection="sentinel-2-l2a",
    item=item.id,
    assets=["B04","B03","B01"], #red, green, coastal
    nodata=0,
)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [ ]:
m = leafmap.Map(zoom=1, google_map="ROADMAP")
m.add_stac_layer(
    collection="sentinel-2-l2a",
    item=item.id,
    assets=["B04", "B03", "B01"], #red, green, coastal
    nodata=0,
    rescale="-1,100", #-1,100 | -100,100 | -1,1
)
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…